# ML-Web: Обучение моделей анализа тональности

Этот ноутбук предназначен для работы в Google Colab и позволяет:
- Загружать данные для обучения
- Обучать baseline модель (TF-IDF + Logistic Regression)
- Обучать transformer модель (RuBERT)
- Оценивать качество моделей
- Сохранять результаты


In [ ]:
# Установка зависимостей
!pip install -q pandas numpy scikit-learn joblib transformers datasets evaluate torch accelerate


## 1. Загрузка данных

Загрузите ваш CSV файл с данными. Файл должен содержать колонки:
- `ID` - уникальный идентификатор записи
- `text` - текст отзыва
- `src` - источник данных (откуда взят отзыв)
- `label` - метка класса (0 - neutral, 1 - positive, 2 - negative)


In [ ]:
from google.colab import files
import pandas as pd
from pathlib import Path
import os

# Создаем директории
os.makedirs('data', exist_ok=True)
os.makedirs('models', exist_ok=True)
os.makedirs('reports', exist_ok=True)

# Загрузка файла (раскомментируйте, если нужно загрузить файл)
# uploaded = files.upload()
# for filename in uploaded.keys():
#     os.rename(filename, f'data/{filename}')

# Или используйте файл из Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# # Скопируйте файл: !cp /content/drive/MyDrive/path/to/train.csv data/train.csv

# Загрузите данные
DATA_PATH = 'data/train.csv'  # Измените путь при необходимости

# Проверка данных
if os.path.exists(DATA_PATH):
    df = pd.read_csv(DATA_PATH)
    print(f"Загружено строк: {len(df)}")
    print(f"Колонки: {df.columns.tolist()}")
    
    # Проверка наличия всех необходимых колонок
    required_columns = ['ID', 'text', 'src', 'label']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"\n⚠️ Внимание: отсутствуют колонки: {missing_columns}")
    else:
        print("\n✅ Все необходимые колонки присутствуют")
    
    print(f"\nРаспределение классов (label):")
    print(df['label'].value_counts().sort_index())
    
    if 'src' in df.columns:
        print(f"\nРаспределение по источникам (src):")
        print(df['src'].value_counts())
        print(f"\nУникальных источников: {df['src'].nunique()}")
    
    print(f"\nПримеры данных:")
    print(df.head())
    
    # Сохраняем информацию о датасете
    dataset_info = {
        "total_rows": len(df),
        "columns": df.columns.tolist(),
        "label_distribution": df['label'].value_counts().to_dict() if 'label' in df.columns else {},
        "src_distribution": df['src'].value_counts().to_dict() if 'src' in df.columns else {},
        "unique_sources": int(df['src'].nunique()) if 'src' in df.columns else 0,
    }
    
    with open('reports/dataset_info.json', 'w', encoding='utf-8') as f:
        import json
        json.dump(dataset_info, f, indent=2, ensure_ascii=False)
    print(f"\nИнформация о датасете сохранена в reports/dataset_info.json")
else:
    print(f"Файл {DATA_PATH} не найден. Загрузите файл выше.")


## 1.1 Очистка данных

Очищаем текст от лишнего: HTML-теги, ссылки, лишние пробелы, спецсимволы и т.д.


In [ ]:
import re
import html

def clean_text(text):
    """
    Очистка текста от лишних символов и нормализация.
    """
    if not isinstance(text, str):
        text = str(text)
    
    # 1. Декодируем HTML-сущности
    text = html.unescape(text)
    
    # 2. Удаляем HTML-теги
    text = re.sub(r'<[^>]+>', ' ', text)
    
    # 3. Удаляем URL-адреса
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    
    # 4. Удаляем email-адреса
    text = re.sub(r'\S+@\S+\.\S+', ' ', text)
    
    # 5. Удаляем номера телефонов (разные форматы)
    text = re.sub(r'[\+]?[78]?[\s\-]?\(?\d{3}\)?[\s\-]?\d{3}[\s\-]?\d{2}[\s\-]?\d{2}', ' ', text)
    
    # 6. Удаляем специальные символы и знаки пунктуации, оставляя только буквы, цифры и базовую пунктуацию
    text = re.sub(r'[^\w\s\.\,\!\?\-\:\;\(\)]+', ' ', text)
    
    # 7. Заменяем множественные пробелы на один
    text = re.sub(r'\s+', ' ', text)
    
    # 8. Удаляем пробелы в начале и конце
    text = text.strip()
    
    # 9. Приводим к нижнему регистру (опционально, можно отключить для transformer-моделей)
    # text = text.lower()
    
    return text

def remove_duplicates(df, subset=['text'], keep='first'):
    """
    Удаление дубликатов по указанным колонкам.
    """
    original_size = len(df)
    df_cleaned = df.drop_duplicates(subset=subset, keep=keep)
    removed = original_size - len(df_cleaned)
    print(f"Удалено дубликатов: {removed} из {original_size} ({removed/original_size*100:.2f}%)")
    return df_cleaned

def remove_short_texts(df, min_length=10, text_column='text'):
    """
    Удаление слишком коротких текстов.
    """
    original_size = len(df)
    df_cleaned = df[df[text_column].str.len() >= min_length]
    removed = original_size - len(df_cleaned)
    print(f"Удалено коротких текстов (< {min_length} символов): {removed} из {original_size} ({removed/original_size*100:.2f}%)")
    return df_cleaned

def remove_empty_or_nan(df, text_column='text'):
    """
    Удаление строк с пустыми или NaN текстами.
    """
    original_size = len(df)
    df_cleaned = df.dropna(subset=[text_column])
    df_cleaned = df_cleaned[df_cleaned[text_column].str.strip() != '']
    removed = original_size - len(df_cleaned)
    print(f"Удалено пустых/NaN текстов: {removed} из {original_size} ({removed/original_size*100:.2f}%)")
    return df_cleaned

# =============================================================================
# ОЧИСТКА ДАННЫХ
# =============================================================================

if os.path.exists(DATA_PATH):
    print("=" * 60)
    print("ОЧИСТКА ДАННЫХ")
    print("=" * 60)
    
    # Загружаем данные
    df = pd.read_csv(DATA_PATH)
    print(f"\n📊 Исходный размер: {len(df)} строк")
    
    # 1. Удаляем пустые и NaN
    df = remove_empty_or_nan(df)
    
    # 2. Очищаем текст
    print("\n🧹 Очистка текстов...")
    df['text_original'] = df['text'].copy()  # Сохраняем оригинал для сравнения
    df['text'] = df['text'].apply(clean_text)
    
    # 3. Удаляем слишком короткие тексты
    df = remove_short_texts(df, min_length=10)
    
    # 4. Удаляем дубликаты
    df = remove_duplicates(df, subset=['text'])
    
    print(f"\n✅ Итоговый размер: {len(df)} строк")
    
    # Показываем примеры очистки
    print("\n📝 Примеры очистки текста:")
    sample_indices = df.head(3).index
    for idx in sample_indices:
        orig = df.loc[idx, 'text_original']
        clean = df.loc[idx, 'text']
        if orig != clean:
            print(f"\nОригинал: {orig[:100]}...")
            print(f"Очищенный: {clean[:100]}...")
    
    # Удаляем временную колонку
    df = df.drop(columns=['text_original'])
    
    # Сохраняем очищенные данные
    CLEAN_DATA_PATH = 'data/train_clean.csv'
    df.to_csv(CLEAN_DATA_PATH, index=False)
    print(f"\n💾 Очищенные данные сохранены: {CLEAN_DATA_PATH}")
    
    # Обновляем путь к данным для использования очищенных данных
    DATA_PATH = CLEAN_DATA_PATH
    print(f"\n🔄 Для обучения будет использоваться: {DATA_PATH}")
else:
    print(f"⚠️ Файл {DATA_PATH} не найден. Сначала загрузите данные.")


## 2. Обучение Baseline модели (TF-IDF + Logistic Regression)


In [ ]:
import json
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# Загрузка данных
df = pd.read_csv(DATA_PATH)
texts = df['text'].astype(str)
labels = df['label']

# Сохраняем информацию о источниках и ID для анализа
if 'src' in df.columns:
    srcs = df['src'].astype(str)
    print(f"Источники данных: {srcs.value_counts().to_dict()}")
else:
    srcs = None

if 'ID' in df.columns:
    ids = df['ID']
    print(f"Диапазон ID: {ids.min()} - {ids.max()}")
else:
    ids = None

# Маппинг меток: 0 -> neutral, 1 -> positive, 2 -> negative
label_mapping = {0: "neutral", 1: "positive", 2: "negative"}
labels = labels.map(label_mapping).fillna(labels.astype(str))

# Разделение на train/test (сохраняем индексы для отслеживания источников)
train_indices, test_indices = train_test_split(
    df.index, test_size=0.2, random_state=42, stratify=labels
)

X_train = texts.iloc[train_indices]
X_test = texts.iloc[test_indices]
y_train = labels.iloc[train_indices]
y_test = labels.iloc[test_indices]

# Информация о разделении
if srcs is not None:
    train_srcs = srcs.iloc[train_indices]
    test_srcs = srcs.iloc[test_indices]
    print(f"\nРаспределение источников в train:")
    print(train_srcs.value_counts())
    print(f"\nРаспределение источников в test:")
    print(test_srcs.value_counts())

print(f"\nTrain: {len(X_train)}, Test: {len(X_test)}")


In [ ]:
# =============================================================================
# НАСТРОЙКИ BASELINE МОДЕЛИ
# =============================================================================

# Параметры TF-IDF
TFIDF_NGRAM_RANGE = (1, 2)  # Униграммы и биграммы
TFIDF_MIN_DF = 1            # Минимальная частота слова
TFIDF_MAX_DF = 0.95         # Максимальная частота слова
TFIDF_SUBLINEAR_TF = True   # Использовать субнормализацию TF

# Параметры Logistic Regression
LR_C = 1.0                  # Обратная сила регуляризации (чем меньше, тем сильнее регуляризация)
                            # C ~ 1/learning_rate в градиентных методах
LR_MAX_ITER = 5000          # Максимальное число итераций
LR_CLASS_WEIGHT = 'balanced' # Балансировка весов классов: 'balanced', None, или словарь
                            # 'balanced' автоматически подстраивает веса обратно пропорционально частоте классов
LR_SOLVER = 'lbfgs'         # Алгоритм оптимизации: 'lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga'

print("⚙️ Настройки Baseline модели:")
print(f"  TF-IDF ngram_range: {TFIDF_NGRAM_RANGE}")
print(f"  TF-IDF min_df: {TFIDF_MIN_DF}")
print(f"  TF-IDF max_df: {TFIDF_MAX_DF}")
print(f"  LR C (regularization): {LR_C}")
print(f"  LR max_iter: {LR_MAX_ITER}")
print(f"  LR class_weight: {LR_CLASS_WEIGHT}")
print(f"  LR solver: {LR_SOLVER}")

# Создание pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        ngram_range=TFIDF_NGRAM_RANGE,
        min_df=TFIDF_MIN_DF,
        max_df=TFIDF_MAX_DF,
        sublinear_tf=TFIDF_SUBLINEAR_TF,
    )),
    ('clf', LogisticRegression(
        C=LR_C,
        max_iter=LR_MAX_ITER,
        multi_class='auto',
        class_weight=LR_CLASS_WEIGHT,
        solver=LR_SOLVER,
        random_state=42,
    )),
])

# Обучение
print("\n🚀 Обучение baseline модели...")
pipeline.fit(X_train, y_train)

# Предсказания
y_pred = pipeline.predict(X_test)

# Метрики
report = classification_report(y_test, y_pred, output_dict=True, digits=4)
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, digits=4))


In [ ]:
# Сохранение модели
MODEL_PATH = 'models/baseline.joblib'
METADATA_PATH = 'models/metadata.json'

joblib.dump(pipeline, MODEL_PATH)

# Собираем информацию о датасете для метаданных
dataset_metadata = {
    "total_rows": len(df),
    "train_size": len(X_train),
    "test_size": len(X_test),
}

if srcs is not None:
    dataset_metadata["sources"] = {
        "train": srcs.iloc[train_indices].value_counts().to_dict(),
        "test": srcs.iloc[test_indices].value_counts().to_dict(),
        "all": srcs.value_counts().to_dict(),
    }
    dataset_metadata["unique_sources"] = int(srcs.nunique())

metadata = {
    "model_path": str(MODEL_PATH),
    "algorithm": "LogisticRegression",
    "vectorizer": "TfidfVectorizer",
    "classes": sorted(labels.unique().tolist()),
    "test_size": 0.2,
    "random_state": 42,
    "metrics": report,
    "dataset": dataset_metadata,
}

with open(METADATA_PATH, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print(f"Модель сохранена: {MODEL_PATH}")
print(f"Метаданные сохранены: {METADATA_PATH}")
print(f"\nИнформация о датасете включена в метаданные:")
print(f"  - Всего строк: {dataset_metadata['total_rows']}")
if srcs is not None:
    print(f"  - Уникальных источников: {dataset_metadata['unique_sources']}")


## 3. Обучение Transformer модели (RuBERT)

⚠️ **Внимание**: Обучение трансформера может занять много времени и требует GPU в Colab.


In [ ]:
# Проверка доступности GPU
import torch
print(f"CUDA доступна: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Устройство: {torch.cuda.get_device_name(0)}")
    print(f"Память GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")


In [ ]:
import numpy as np
from datasets import Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
)
from evaluate import load as load_metric
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight

# =============================================================================
# НАСТРОЙКИ TRANSFORMER МОДЕЛИ
# =============================================================================

# Модель
MODEL_NAME = "cointegrated/rubert-tiny"

# Директория для сохранения
OUTPUT_DIR = "models/transformer"

# Параметры токенизации
MAX_LENGTH = 256  # Максимальная длина токенов (256-512 для большинства текстов)

# Параметры обучения
EPOCHS = 5                    # Количество эпох (3-10)
BATCH_SIZE = 16               # Размер батча (8-32, зависит от GPU памяти)
LEARNING_RATE = 2e-5          # Learning rate (1e-5 - 5e-5 для fine-tuning BERT)
WEIGHT_DECAY = 0.01           # Weight decay для регуляризации (0.01 - 0.1)
WARMUP_RATIO = 0.1            # Доля шагов для warmup (0.0 - 0.2)

# Early Stopping настройки
EARLY_STOPPING_ENABLED = True       # Включить/выключить early stopping
EARLY_STOPPING_PATIENCE = 2         # Количество эпох без улучшения до остановки
EARLY_STOPPING_THRESHOLD = 0.0      # Минимальное улучшение для считания прогресса

# Балансировка классов
USE_CLASS_WEIGHTS = True      # Включить балансировку весов классов

print("=" * 60)
print("⚙️ НАСТРОЙКИ TRANSFORMER МОДЕЛИ")
print("=" * 60)
print(f"\n📦 Модель: {MODEL_NAME}")
print(f"\n🔧 Параметры обучения:")
print(f"  Эпохи: {EPOCHS}")
print(f"  Размер батча: {BATCH_SIZE}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Weight decay: {WEIGHT_DECAY}")
print(f"  Warmup ratio: {WARMUP_RATIO}")
print(f"\n⏹️ Early Stopping:")
print(f"  Включен: {EARLY_STOPPING_ENABLED}")
if EARLY_STOPPING_ENABLED:
    print(f"  Patience: {EARLY_STOPPING_PATIENCE} эпох")
    print(f"  Threshold: {EARLY_STOPPING_THRESHOLD}")
print(f"\n⚖️ Балансировка классов: {USE_CLASS_WEIGHTS}")


In [ ]:
# Подготовка данных
df = pd.read_csv(DATA_PATH)
texts = df['text'].astype(str)
srcs = df['src'].astype(str)
labels = df['label']

# Маппинг меток
label_mapping = {0: "neutral", 1: "positive", 2: "negative"}
labels = labels.map(label_mapping).fillna(labels.astype(str))

# Создание маппинга классов
expected_classes = ["neutral", "positive", "negative"]
unique_labels = set(labels.unique().tolist())
label2id = {}
for idx, cls in enumerate(expected_classes):
    if cls in unique_labels:
        label2id[cls] = idx

# Разделение на train/val
train_df, val_df = train_test_split(
    pd.DataFrame({"text": texts, "label": labels.map(label2id), "src": srcs}),
    test_size=0.2,
    stratify=labels.map(label2id),
    random_state=42,
)

train_dataset = Dataset.from_pandas(train_df, preserve_index=False)
val_dataset = Dataset.from_pandas(val_df, preserve_index=False)

print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}")
print(f"Классы: {label2id}")


In [ ]:
# Токенизация
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length",
    )

train_tokenized = train_dataset.map(tokenize_function, batched=True)
val_tokenized = val_dataset.map(tokenize_function, batched=True)

train_tokenized = train_tokenized.remove_columns(["text"])
val_tokenized = val_tokenized.remove_columns(["text"])
train_tokenized.set_format("torch")
val_tokenized.set_format("torch")

print("Токенизация завершена")


In [ ]:
# Создание модели
id2label = {idx: label for label, idx in label2id.items()}

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id,
)

print(f"Модель создана: {len(label2id)} классов")


In [ ]:
# Функция для вычисления метрик
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy_metric = load_metric("accuracy")
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    macro_f1 = f1_score(labels, predictions, average="macro")
    return {"accuracy": accuracy, "macro_f1": macro_f1}


In [ ]:
# =============================================================================
# ВЫЧИСЛЕНИЕ ВЕСОВ КЛАССОВ ДЛЯ БАЛАНСИРОВКИ
# =============================================================================

class_weights = None
if USE_CLASS_WEIGHTS:
    # Вычисляем веса классов на основе распределения в train
    train_labels = train_df['label'].values
    unique_classes = np.unique(train_labels)
    class_weights_array = compute_class_weight(
        class_weight='balanced',
        classes=unique_classes,
        y=train_labels
    )
    class_weights = torch.tensor(class_weights_array, dtype=torch.float32)
    
    # Переносим на GPU если доступна
    if torch.cuda.is_available():
        class_weights = class_weights.cuda()
    
    print("⚖️ Веса классов для балансировки:")
    for cls, weight in zip(unique_classes, class_weights_array):
        label_name = id2label.get(cls, str(cls))
        print(f"  {label_name} (id={cls}): {weight:.4f}")

# =============================================================================
# КАСТОМНЫЙ TRAINER С WEIGHTED LOSS
# =============================================================================

class WeightedTrainer(Trainer):
    """Trainer с поддержкой weighted cross-entropy loss для балансировки классов."""
    
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights
    
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        if self.class_weights is not None:
            loss_fn = torch.nn.CrossEntropyLoss(weight=self.class_weights)
        else:
            loss_fn = torch.nn.CrossEntropyLoss()
        
        loss = loss_fn(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss

# =============================================================================
# НАСТРОЙКА TRAINING ARGUMENTS
# =============================================================================

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",       # Оценка каждую эпоху
    save_strategy="epoch",       # Сохранение каждую эпоху
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,   # Warmup для learning rate
    load_best_model_at_end=True, # Загружать лучшую модель в конце
    metric_for_best_model="macro_f1",
    greater_is_better=True,      # Для macro_f1 больше = лучше
    logging_steps=50,
    save_total_limit=3,          # Сохранять только последние 3 чекпоинта
    fp16=torch.cuda.is_available(),  # FP16 для ускорения на GPU
    dataloader_num_workers=2,
    report_to="none",            # Отключаем логирование в wandb и т.п.
)

print("\n🔧 Training Arguments настроены:")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Weight decay: {training_args.weight_decay}")
print(f"  Warmup ratio: {training_args.warmup_ratio}")
print(f"  FP16: {training_args.fp16}")

# =============================================================================
# НАСТРОЙКА CALLBACKS (EARLY STOPPING)
# =============================================================================

callbacks = []
if EARLY_STOPPING_ENABLED:
    early_stopping_callback = EarlyStoppingCallback(
        early_stopping_patience=EARLY_STOPPING_PATIENCE,
        early_stopping_threshold=EARLY_STOPPING_THRESHOLD,
    )
    callbacks.append(early_stopping_callback)
    print(f"\n⏹️ Early Stopping включен:")
    print(f"  Patience: {EARLY_STOPPING_PATIENCE} эпох")
    print(f"  Threshold: {EARLY_STOPPING_THRESHOLD}")

# =============================================================================
# СОЗДАНИЕ TRAINER
# =============================================================================

# Используем WeightedTrainer если включена балансировка классов
if USE_CLASS_WEIGHTS and class_weights is not None:
    trainer = WeightedTrainer(
        class_weights=class_weights,
        model=model,
        args=training_args,
        train_dataset=train_tokenized,
        eval_dataset=val_tokenized,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=callbacks,
    )
    print("\n✅ Используется WeightedTrainer с балансировкой классов")
else:
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized,
        eval_dataset=val_tokenized,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=callbacks,
    )
    print("\n✅ Используется стандартный Trainer")

print("\n🚀 Настройка завершена, начинаем обучение...")


In [ ]:
# Обучение
trainer.train()


In [ ]:
# Оценка модели
eval_metrics = trainer.evaluate()
print("\nМетрики на валидации:")
for key, value in eval_metrics.items():
    print(f"{key}: {value:.4f}")


In [ ]:
# Сохранение модели
os.makedirs(OUTPUT_DIR, exist_ok=True)
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# Собираем информацию о датасете
dataset_metadata = {
    "total_rows": len(df),
    "train_size": len(train_dataset),
    "val_size": len(val_dataset),
}

if srcs is not None:
    dataset_metadata["sources"] = {
        "train": train_df['src'].value_counts().to_dict() if 'src' in train_df.columns else {},
        "val": val_df['src'].value_counts().to_dict() if 'src' in val_df.columns else {},
        "all": srcs.value_counts().to_dict(),
    }
    dataset_metadata["unique_sources"] = int(srcs.nunique())

# Сохранение метаданных с полной информацией о настройках
metadata = {
    "model_type": "transformer",
    "base_model": MODEL_NAME,
    "model_dir": OUTPUT_DIR,
    "classes": list(label2id.keys()),
    "label2id": label2id,
    "id2label": {str(idx): label for label, idx in label2id.items()},
    "max_length": MAX_LENGTH,
    "metrics": eval_metrics,
    "test_size": 0.2,
    "random_state": 42,
    "dataset": dataset_metadata,
    # Новые параметры обучения
    "training_config": {
        "learning_rate": LEARNING_RATE,
        "weight_decay": WEIGHT_DECAY,
        "warmup_ratio": WARMUP_RATIO,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "fp16": torch.cuda.is_available(),
    },
    "early_stopping": {
        "enabled": EARLY_STOPPING_ENABLED,
        "patience": EARLY_STOPPING_PATIENCE if EARLY_STOPPING_ENABLED else None,
        "threshold": EARLY_STOPPING_THRESHOLD if EARLY_STOPPING_ENABLED else None,
    },
    "class_weights": {
        "enabled": USE_CLASS_WEIGHTS,
        "weights": {id2label.get(i, str(i)): float(w) for i, w in enumerate(class_weights_array)} if USE_CLASS_WEIGHTS and class_weights is not None else None,
    },
}

metadata_path = os.path.join(OUTPUT_DIR, "metadata.json")
with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

# Сохранение метрик
report_path = "reports/transformer_metrics.json"
with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(eval_metrics, f, indent=2, ensure_ascii=False)

print(f"\n💾 Модель сохранена: {OUTPUT_DIR}")
print(f"📄 Метаданные сохранены: {metadata_path}")
print(f"📊 Метрики сохранены: {report_path}")
print(f"\n📋 Информация о датасете включена в метаданные:")
print(f"  - Всего строк: {dataset_metadata['total_rows']}")
if srcs is not None:
    print(f"  - Уникальных источников: {dataset_metadata['unique_sources']}")
print(f"\n⚙️ Конфигурация обучения сохранена:")
print(f"  - Learning rate: {LEARNING_RATE}")
print(f"  - Early stopping: {'включен' if EARLY_STOPPING_ENABLED else 'выключен'}")
print(f"  - Class weights: {'включены' if USE_CLASS_WEIGHTS else 'выключены'}")


## 4. Визуализация результатов


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Загрузка метрик baseline
with open(METADATA_PATH, 'r', encoding='utf-8') as f:
    baseline_metrics = json.load(f)

# Загрузка данных для анализа источников
df_full = pd.read_csv(DATA_PATH)

# Сравнение метрик моделей
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Baseline метрики
baseline_metrics_dict = baseline_metrics.get('metrics', {})
if 'accuracy' in baseline_metrics_dict:
    baseline_acc = baseline_metrics_dict['accuracy']
    baseline_f1 = baseline_metrics_dict.get('macro avg', {}).get('f1-score', 0)
else:
    baseline_acc = 0
    baseline_f1 = 0

# Transformer метрики
transformer_acc = eval_metrics.get('eval_accuracy', 0)
transformer_f1 = eval_metrics.get('eval_macro_f1', 0)

# График сравнения Accuracy
axes[0, 0].bar(['Baseline', 'Transformer'], [baseline_acc, transformer_acc], color=['skyblue', 'lightcoral'])
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].set_title('Сравнение Accuracy')
axes[0, 0].set_ylim([0, 1])
for i, v in enumerate([baseline_acc, transformer_acc]):
    axes[0, 0].text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom')

# График сравнения F1
axes[0, 1].bar(['Baseline', 'Transformer'], [baseline_f1, transformer_f1], color=['skyblue', 'lightcoral'])
axes[0, 1].set_ylabel('Macro F1')
axes[0, 1].set_title('Сравнение Macro F1')
axes[0, 1].set_ylim([0, 1])
for i, v in enumerate([baseline_f1, transformer_f1]):
    axes[0, 1].text(i, v + 0.01, f'{v:.4f}', ha='center', va='bottom')

# Распределение по источникам (если есть)
if 'src' in df_full.columns:
    src_counts = df_full['src'].value_counts()
    axes[1, 0].bar(range(len(src_counts)), src_counts.values, color='steelblue')
    axes[1, 0].set_xticks(range(len(src_counts)))
    axes[1, 0].set_xticklabels(src_counts.index, rotation=45, ha='right')
    axes[1, 0].set_ylabel('Количество записей')
    axes[1, 0].set_title('Распределение по источникам (src)')
    for i, v in enumerate(src_counts.values):
        axes[1, 0].text(i, v + max(src_counts.values) * 0.01, str(v), ha='center', va='bottom')

# Распределение классов по источникам (если есть)
if 'src' in df_full.columns and 'label' in df_full.columns:
    label_mapping = {0: "neutral", 1: "positive", 2: "negative"}
    df_full['label_name'] = df_full['label'].map(label_mapping)
    pivot_data = pd.crosstab(df_full['src'], df_full['label_name'])
    pivot_data.plot(kind='bar', ax=axes[1, 1], color=['#ff9999', '#66b3ff', '#99ff99'])
    axes[1, 1].set_ylabel('Количество записей')
    axes[1, 1].set_title('Распределение классов по источникам')
    axes[1, 1].legend(title='Класс', bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[1, 1].tick_params(axis='x', rotation=45)
else:
    axes[1, 1].text(0.5, 0.5, 'Нет данных об источниках', 
                     ha='center', va='center', transform=axes[1, 1].transAxes)
    axes[1, 1].set_title('Распределение классов по источникам')

plt.tight_layout()
plt.show()

print(f"Baseline - Accuracy: {baseline_acc:.4f}, Macro F1: {baseline_f1:.4f}")
print(f"Transformer - Accuracy: {transformer_acc:.4f}, Macro F1: {transformer_f1:.4f}")

if 'src' in df_full.columns:
    print(f"\nСтатистика по источникам:")
    print(df_full['src'].value_counts())
    if 'label' in df_full.columns:
        print(f"\nРаспределение классов по источникам:")
        print(pd.crosstab(df_full['src'], df_full['label_name']))


## 5. Сохранение результатов

Скачайте обученные модели и метрики:


In [ ]:
# Создание архива с моделями
import shutil

# Создаем архив
shutil.make_archive('ml_web_models', 'zip', 'models')
shutil.make_archive('ml_web_reports', 'zip', 'reports')

print("Архивы созданы:")
print("- ml_web_models.zip (модели)")
print("- ml_web_reports.zip (отчеты)")

# Скачать файлы
files.download('ml_web_models.zip')
files.download('ml_web_reports.zip')


## 6. Тестирование модели

Протестируйте обученную модель на примерах:


In [ ]:
# Загрузка обученной модели
from transformers import pipeline

# Выберите модель для тестирования
USE_TRANSFORMER = True  # False для baseline

if USE_TRANSFORMER and os.path.exists(OUTPUT_DIR):
    # Transformer модель
    classifier = pipeline(
        "text-classification",
        model=OUTPUT_DIR,
        tokenizer=OUTPUT_DIR,
        device=0 if torch.cuda.is_available() else -1,
    )
    print("Загружена transformer модель")
else:
    # Baseline модель
    pipeline_model = joblib.load(MODEL_PATH)
    print("Загружена baseline модель")

# Примеры для тестирования
test_texts = [
    "Отличный сервис, все быстро и качественно!",
    "Плохое обслуживание, долго ждал ответа.",
    "Обычный день, ничего особенного.",
]

print("\nТестирование модели:")
print("=" * 60)

if USE_TRANSFORMER and os.path.exists(OUTPUT_DIR):
    for text in test_texts:
        result = classifier(text)[0]
        print(f"\nТекст: {text}")
        print(f"Класс: {result['label']}")
        print(f"Уверенность: {result['score']:.4f}")
else:
    for text in test_texts:
        proba = pipeline_model.predict_proba([text])[0]
        classes = pipeline_model.classes_
        pred = pipeline_model.predict([text])[0]
        print(f"\nТекст: {text}")
        print(f"Класс: {pred}")
        for cls, score in zip(classes, proba):
            print(f"  {cls}: {score:.4f}")


## 📝 Полезные советы

### Оптимизация для больших датасетов:
- Уменьшите `EPOCHS` для быстрого тестирования
- Увеличьте `BATCH_SIZE` если есть GPU с большей памятью
- Используйте `save_total_limit=1` чтобы экономить место

### Альтернативные модели:
- `cointegrated/rubert-tiny` - быстрая, легкая модель
- `cointegrated/rubert-base` - более точная, но медленнее
- `DeepPavlov/rubert-base-cased-conversational` - для разговорной речи

### Сохранение в Google Drive:
```python
from google.colab import drive
drive.mount('/content/drive')
!cp -r models /content/drive/MyDrive/ml_web_models
!cp -r reports /content/drive/MyDrive/ml_web_reports
```

### Загрузка обученной модели:
После обучения вы можете скачать архивы `ml_web_models.zip` и `ml_web_reports.zip` 
и использовать их в вашем локальном проекте ML-Web.
